In [1]:
import pandas as pd
import numpy as np

import networkx as nx
from networkx.algorithms.community import k_clique_communities
import random as rd
rd.seed(42)

import warnings
warnings.filterwarnings('ignore')

## Random Walk Sampling Method 2
### Sample 7

In [2]:
G_rwm2_7 = nx.read_gml(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\Samples\Random Walk Sampling\G_rw2_7.gml')

In [3]:
# Confirm that Graph is Directed
nx.is_directed(G_rwm2_7)

True

In [4]:
# Display Size of the Graph
print(nx.info(G_rwm2_7))

MultiDiGraph with 50000 nodes and 135896 edges


Confirmed that the graph meets the criterea of having more edges than nodes and maintained the directed structure.

In [5]:
# List all nodes in the sample
n_rwm2_7 = list(G_rwm2_7.nodes)

In [6]:
# Get node properties from data exploration exercise for comparison
node_properties = pd.read_csv(r'C:\Users\sarah\Documents\UNI\Masters\Study\Term_3\Master Project\node_properties_cl.csv')
node_properties[['degree', 'in_degree', 'out_degree', 'eigen_centrality']].describe()

,degree,in_degree,out_degree,eigen_centrality
count,4.666873e+06,4.666873e+06,4.666873e+06,4.666873e+06
mean,4.494298e+00,2.247149e+00,2.247149e+00,3.970074e-05
std,3.935922e+01,1.998412e+01,2.868249e+01,4.611943e-04
min,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
25%,1.000000e+00,0.000000e+00,0.000000e+00,1.193322e-21
50%,2.000000e+00,1.000000e+00,1.000000e+00,4.892620e-20
75%,3.000000e+00,1.000000e+00,2.000000e+00,1.517475e-14
max,2.791700e+04,1.211600e+04,2.533500e+04,1.642653e-01


In [7]:
rwm2_7_df = nx.to_pandas_edgelist(G_rwm2_7) 
rwm2_7_df.head()

,source,target,weight
0,13xEuCvTPsFNQJkoL4JfCvSyXYt4xdpD9L,162QCBFdyFwdaYdqy7NXd93zDCYWaMhPHd,1.0
1,13xEuCvTPsFNQJkoL4JfCvSyXYt4xdpD9L,18yDipuzMszsX8eFQ7pGPLmGuq9beboGii,1.0
2,33MAkwCKniBK8QDML5ojP8cY5fFeiVEjbv,32jPMzNzHKE2f6u6V1Ch9qxXXf456D83F5,1.0
3,1t1wgvtPK6Ei4hpS31YxTseN4metDUids,17hLZbf8erC9ePFEkHCcdsQMy4gZKjp8EL,1.0
4,1t1wgvtPK6Ei4hpS31YxTseN4metDUids,1AMQiPsM3rXNiD9buAiGG9N1Ebtr9UCkJW,1.0


In [8]:
# Number of unique input addresses
rwm2_7_df['source'].nunique()

31940

In [9]:
# Number of unique output addresses
rwm2_7_df['target'].nunique()

31141

We observe relative balance between input and output addresses, with slightly more input addresses. In the full dataset there is also relative balance, but there were more output addresses. 

In [10]:
rwm2_7_df.describe()

,weight
count,135896.000000
mean,0.776984
std,0.467355
min,-69.648093
25%,0.516694
50%,1.000000
75%,1.000000
max,52.821905


The weight variable shows that the distribution of the weighting (amount*0.5 + timestamp*0.5) is similar to that of overall transaction amounts.

In [11]:
# Subset by fraudulent/high-risk nodes
fr_nodes = node_properties[node_properties['fraud_flag'] == 1]
fr_nodes = fr_nodes['address'].tolist()
print('Total number of fraudulent/high-risk nodes : ', len(fr_nodes))

# Check for number of fraudulent/high-risk addresses in the sample
print('Number of fraudulent/high-risk nodes in sample : ', len(set(n_rwm2_7).intersection(fr_nodes)))

Total number of fraudulent/high-risk nodes :  4563
Number of fraudulent/high-risk nodes in sample :  262


These numbers suggest that random selection of nodes worsens the class imbalance problem, but when considering the counts relative to the size of the network the issue is less pronounced.

Full Network: 0.11% 

Sample: 0.52%

## Node Properties

In [12]:
# Get Properties

address = [node for (node, val) in G_rwm2_7.degree()]
degree = [val for (node, val) in G_rwm2_7.degree()]
in_degree = [val for (node, val) in G_rwm2_7.in_degree()]
out_degree = [val for (node, val) in G_rwm2_7.out_degree()]
#eigen_centrality = nx.eigenvector_centrality(G_rwm2_7) # NetworkXNotImplemented: not implemented for multigraph type
#eigen_centrality = [eigen_centrality[node] for node in eigen_centrality]

In [13]:
# Add Properties to DataFrame

nodes = pd.DataFrame()
nodes['address'] = address
nodes['degree'] = degree
nodes['in_degree'] = in_degree
nodes['out_degree'] = out_degree
#nodes['eigen_centrality'] = eigen_centrality

In [14]:
# Add fraud flag

nodes = nodes.assign(**dict.fromkeys(['fraud_flag'], 0))

for i in fr_nodes:
    nodes.loc[nodes.address == i, 'fraud_flag'] = 1
    
nodes.head()

,address,degree,in_degree,out_degree,fraud_flag
0,13xEuCvTPsFNQJkoL4JfCvSyXYt4xdpD9L,2,0,2,0
1,12t97nWmHYg5UDnsL2nizTEbTNLF7xk4FV,4,4,0,0
2,33MAkwCKniBK8QDML5ojP8cY5fFeiVEjbv,5,4,1,0
3,1t1wgvtPK6Ei4hpS31YxTseN4metDUids,7,3,4,0
4,13qAtDZ7QPWzkZ7gD2N6tEscmHnzDpoXp6,1,1,0,0


In [15]:
# Describe sample properties overall

nodes[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,50000.000000,50000.000000,50000.000000
mean,5.435840,2.717920,2.717920
std,38.155099,17.059608,24.807715
min,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,4037.000000,1433.000000,2617.000000


In [16]:
# Describe Properties of Fraudulent Nodes

nodes_fraud = nodes[nodes['fraud_flag'] == 1]
nodes_fraud[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,262.000000,262.000000,262.000000
mean,144.538168,59.305344,85.232824
std,466.704724,187.595591,308.868016
min,1.000000,0.000000,0.000000
25%,7.250000,2.000000,2.000000
50%,30.000000,10.000000,10.500000
75%,86.500000,41.000000,38.000000
max,4037.000000,1433.000000,2617.000000


In [17]:
# Describe Properties of Non-Fraudulent Nodes

nodes_licit = nodes[nodes['fraud_flag'] == 0]
nodes_licit[['degree', 'in_degree', 'out_degree']].describe()

,degree,in_degree,out_degree
count,49738.000000,49738.000000,49738.000000
mean,4.703104,2.419840,2.283264
std,14.765453,9.535844,9.054886
min,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000
75%,4.000000,2.000000,2.000000
max,984.000000,982.000000,598.000000


For all 3 cases the degree distribution properties are similar to the overall network. One concern is the existence of isolated nodes in the sample.

## Graph Properties

In [18]:
# Density
nx.density(G_rwm2_7) # Connectivity remains very low

5.4359487189743794e-05

In [19]:
# Communities - k-Cliques

com_generator_rwm2_7 = k_clique_communities(G_rwm2_7.to_undirected(), k=10)
com_rwm2_7 = next(com_generator_rwm2_7)
print('The number of communities in the sample : ' + str(len(com_rwm2_7)))

The number of communities in the sample : 11


The sample has a variety of community structures. (Note: Giant Component = 107)